In [9]:
#Класс нейрона
import random
import math
import numpy as np

class Neuron:
    inputW = []
    t = 0.0
    
    def __init__(self, size):
        self.inputW = []
        t = 0.0
        for i in range(size):
            a = random.random()/10.0
            self.inputW.append(a - 0.05)
            #self.inputW.append(0)
        a = random.random()/10.0
        self.t = a - 0.05
        #self.t = 0
    
    def compute(self, inputS):
        S = self.t 
        for i in range(len(inputS)):
            S += self.inputW[i] * inputS[i]
        return self.__activation(S)
        
    def __activation(self, x):
        #return np.tanh(x)
        return self.__sigmoid(x)
        #return self.__stup(x)
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __stup(self, x):
        return -1 if x <= 0 else 1
    
    def getW(self):
        return self.inputW
    
    def getT(self):
        return self.t
    

In [34]:
#Класс сети
import sys

class Net:
    __err = 0
    __alpha = 0
    __neuron = []
    size = 0
    outputN = []
    
    def __init__(self, sizeN, err, alpha):
        self.__alpha = alpha
        self.__err = err
        self.__neuron = []
        self.size = len(sizeN)
        self.output = []
        
        for i in range(1, self.size): 
            temp = []
            for j in range(sizeN[i]):
                temp.append(Neuron(sizeN[i-1]))
            self.__neuron.append(temp)
    
#OLD but golg 
    def err(self, inputL):
        s = []

        for i in inputL:
            output = self.compute(i[:-1])
            s.append(self.learnErr(output[-1], i[-1]))
            #print('input', i)
            #print('output', output)
        #se = sum(s)/len(s)
        sm = max(s)
        #print('Средняя ошибка :', se)
        #return True if se > (1 - self.__err) else False
        return True if sm > (1 - self.__err) else False
    
    # Вычисляет один слой
    def process(self, inp, neur):
        out = []
        for i in neur:
            out.append(i.compute(inp))
        return out
    
    def learnErr(self, y, d):
        res = 0
        nd = []
        for i in range(len(y)):
            nd.append(0) if d != i else nd.append(1)
            res += (nd[i] - y[i]) ** 2
            #res += (d - y[i]) ** 2
        res /= 2
        #print('d', d, 'nd', nd, 'y', y)
        return res
    
    def reversLearnOut(self, y, d):
        gamma = []
        nd = []
        for i in range(len(y)):
            nd.append(0) if d != i else nd.append(1)
            gamma.append((y[i] * (1 - y[i])) * (nd[i] - y[i]))
            #gamma.append((y[i] * (1 - y[i])) * (d - y[i]))
        return gamma
    
    def reversLearnInside(self, out, neur, neurNext, g):
        gamma = []
        for i in range(len(out)):
            
            s = 0
            for n in range(len(neurNext)):    
                s += neurNext[n].inputW[i] * g[n]
                
            gamma.append((out[i] * (1 - out[i])) * (s))
        return gamma
    
    def modification(self, neur, alpha, gamma, inp):
        for n in range(len(neur)):
#             print('len(neur[n].inputW)', len(neur[n].inputW), 'len(inp)', len(inp))
            for i in range(len(neur[n].inputW)):
                
                neur[n].inputW[i] += alpha * gamma[n] * inp[i]
            neur[n].t += alpha * gamma[n] * 1
        return neur
    
    def compute(self, inp):
        output = []
        # Первый слой
        output.append(self.process(inp, self.__neuron[0]))
        # Для каждого следующего слоя
        for n in self.__neuron[1:]:
            # Вычисляем значения нейронов
            output.append(self.process(output[-1], n))
        # output[-1] - результат работы сети
        return output
  
    def printProgress(self, i, err):
        sys.stdout.write("\rСуммарная квадратичная ошибка для образа: {0} {1}".format(i, round(err, 5)))
        sys.stdout.flush()

#OLD but gold
    def learn(self, inputL):
        flag = True
        count = 0
        output = []
        gamma = []
        while(flag):
        #while(count < 3):
            #print('Эпоха:', count)
            count += 1
            flag = False
            
            # Для каждого образа
            for i in inputL:
                
                output = []
                gamma = []
                
                # Вычисляем выход
                output = self.compute(i[:-1])
               
                # Вычисление суммарной квадратичной ошибки
                err = self.learnErr(output[-1], i[-1])
                #print('Суммарная квадратичная ошибка для образа', i, err, end = '\r')
                self.printProgress(i, err)
                #print('Output', output)
                
                # Ошибка гамма
                gamma.append(self.reversLearnOut(output[-1], i[-1]))
                for g in range(0, len(self.__neuron) - 1)[::-1]:
                    gamma.append(self.reversLearnInside(output[g], self.__neuron[g], self.__neuron[g+1], gamma[len(self.__neuron) - 2 -g]))
                gamma = gamma[::-1]
                #print('gamma', gamma)
                

                # Модификация весов
                self.__neuron[0] = self.modification(self.__neuron[0], self.__alpha, gamma[0], i[:-1])
                for n in range(1, len(self.__neuron)):
                    self.__neuron[n] = self.modification(self.__neuron[n], self.__alpha, gamma[n], output[n-1])

            
            flag = self.err(inputL)
        
        print('\nКоличество эпох :', count)

        for i in inputL:
            print('inputL :', i, 'max :', self.compute(i[:-1])[-1].index(max(self.compute(i[:-1])[-1])), 'output :', (list(map(lambda x : round(x, 5), self.compute(i[:-1])[-1]))))
#         print('Веса')
#         for i in range(len(self.__neuron)):
#             print(np.array(self.__neuron[i].inputW))
#         print('\n\n\n')
#         #print(np.array(self.__neuron))
        
    
#OLD    
    def test(self, inputT):
        for i in inputT:
            print('Образ :', i, 'Результат :', self.compute(i[:-1])[-1].index(max(self.compute(i[:-1])[-1])))
        
        

In [30]:
#Класс данных
class Data:
    
    inputL = [
        [2, 0, 0, 0, 2],
        [2, 0, 0, 1, 2],
        [2, 0, 1, 1, 0],
        [2, 0, 1, 2, 0],
        [2, 1, 0, 2, 3],
        [2, 1, 0, 1, 0],
        [1, 0, 0, 0, 2],
        [1, 0, 0, 1, 3],
        [1, 0, 1, 1, 0],
        [1, 0, 1, 2, 3],
        [1, 1, 0, 2, 3],
        [1, 1, 0, 1, 3],
        [0, 0, 0, 0, 2],
        [0, 0, 0, 1, 3],
        [0, 0, 1, 1, 3],
        [0, 0, 1, 2, 1],
        [0, 1, 0, 2, 1],
        [0, 1, 0, 1, 3],
    ]
    
    
#     inputL = [
#         [0,0,0],
#         [0,1,1],
#         [1,0,1],
#         [1,1,0],
#     ]
    
    
    inputT = [
        [2,1,1,1, 0],
        [1,1,1,2, 3],
        [0,0,0,0, 2],
        [0,1,1,1, 3],
        [2,0,1,3, 3],
        [2,1,0,3, 3],
        [0,1,0,3, 1],
    ]
    output = {0 : 'Атаковать', 1 : 'Бежать', 2 : 'Уворачиваться', 3 : 'Прятаться'}

In [35]:
web = Net([4 ,3 ,4], 0.97, 0.5)
# web = Net([2, 4, 2], 0.97, 0.5)
# web = Net([2, 4 ,4 ,2], 0.95, 1)
web.learn(Data.inputL)

Суммарная квадратичная ошибка для образа: [0, 1, 0, 1, 3] 0.00767
Количество эпох : 272
inputL : [2, 0, 0, 0, 2] max : 2 output : [0.05747, 0.00134, 0.9421, 0.05414]
inputL : [2, 0, 0, 1, 2] max : 2 output : [0.12895, 0.00218, 0.86007, 0.0499]
inputL : [2, 0, 1, 1, 0] max : 0 output : [0.93083, 0.0197, 0.08878, 0.02081]
inputL : [2, 0, 1, 2, 0] max : 0 output : [0.83082, 0.02811, 0.01789, 0.10649]
inputL : [2, 1, 0, 2, 3] max : 3 output : [0.09048, 0.04927, 0.00032, 0.95821]
inputL : [2, 1, 0, 1, 0] max : 0 output : [0.86497, 0.01722, 0.08775, 0.03578]
inputL : [1, 0, 0, 0, 2] max : 2 output : [0.05342, 0.0013, 0.9451, 0.05529]
inputL : [1, 0, 0, 1, 3] max : 3 output : [0.00334, 0.0035, 0.1586, 0.8633]
inputL : [1, 0, 1, 1, 0] max : 0 output : [0.90569, 0.01939, 0.08102, 0.02762]
inputL : [1, 0, 1, 2, 3] max : 3 output : [0.07967, 0.07521, 0.00032, 0.9339]
inputL : [1, 1, 0, 2, 3] max : 3 output : [0.06349, 0.07834, 0.00032, 0.9374]
inputL : [1, 1, 0, 1, 3] max : 3 output : [0.05613, 0

In [36]:
web.test(Data.inputT)

Образ : [2, 1, 1, 1, 0] Результат : 0
Образ : [1, 1, 1, 2, 3] Результат : 3
Образ : [0, 0, 0, 0, 2] Результат : 2
Образ : [0, 1, 1, 1, 3] Результат : 3
Образ : [2, 0, 1, 3, 3] Результат : 3
Образ : [2, 1, 0, 3, 3] Результат : 3
Образ : [0, 1, 0, 3, 1] Результат : 1
